Next Word Prediction means predicting the most likely word or phrase that will come next in a sentence or text. It is like having an inbuilt feature on an application that suggests the next word as you type or speak. The Next Word Prediction Models are used in applications like messaging apps, search engines, virtual assistants, and autocorrect features on smartphones. So, if you want to learn how to build a Next Word Prediction Model, this article is for you. In this article, I’ll take you through building a Next Word Prediction Model with Deep Learning using Python.

Next word prediction is a language modelling task in Machine Learning that aims to predict the most 
probable word or sequence of words that follows a given input context. 

This task utilizes statistical patterns and linguistic structures to generate accurate predictions
based on the context provided.

The Next Word Prediction models have a range of applications across various industries. For example, when you start typing a message on your phone, it suggests the next word to speed up your typing. Similarly, search engines predict and show search suggestions as you type in the search bar. Next word prediction helps us communicate faster and more accurately by anticipating what we might say or search for.

To build a Next Word Prediction model:

start by collecting a diverse dataset of text documents, 
preprocess the data by cleaning and tokenizing it, 
prepare the data by creating input-output pairs, 
engineer features such as word embeddings, 
select an appropriate model like an LSTM or GPT, 
train the model on the dataset while adjusting hyperparameters,
improve the model by experimenting with different techniques and architectures.
This iterative process allows businesses to develop accurate and efficient Next Word Prediction models that can be applied in various applications.

So the process of building a Next Word Prediction model starts by collecting textual data that can be a vocabulary for our model. For example, the way you type on your smartphone’s keyboard becomes the vocabulary of the next word prediction model of your smartphone’s keyboard. In the same way, we need textual data for our model. I found an ideal dataset for this task based on the text of a book on Sherlock Holmes. You can download the dataset from here.

Next Word Prediction Model using Python
I hope you now know what a Next Word Prediction model is. In this section, I’ll take you through how to build a Next Word Prediction model using Python and Deep Learning. So, let’s start this task by importing the necessary Python libraries and the dataset

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import warnings
warnings.filterwarnings('ignore')

2025-06-10 20:25:12.400749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749587112.889524      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749587113.017095      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
with open('/kaggle/input/book-data/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
#tokenizing the text to create a sequence of words

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In the above code, the text is tokenized, which means it is divided into individual words or tokens. The ‘Tokenizer’ object is created, which will handle the tokenization process. The ‘fit_on_texts’ method of the tokenizer is called, passing the ‘text’ as input. This method analyzes the text and builds a vocabulary of unique words, assigning each word a numerical index. The ‘total_words’ variable is then assigned the value of the length of the word index plus one, representing the total number of distinct words in the text.

Now let’s create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequences = token_list[:i+1]
        input_sequences.append(n_gram_sequences)

In [5]:
input_sequences[0:10]

[[1, 1561],
 [1, 1561, 5],
 [1, 1561, 5, 129],
 [1, 1561, 5, 129, 34],
 [647, 4498],
 [647, 4498, 4499],
 [226, 5],
 [226, 5, 1562],
 [6, 827],
 [6, 827, 7]]

In the above code, the text data is split into lines using the ‘\n’ character as a delimiter. For each line in the text, the ‘texts_to_sequences’ method of the tokenizer is used to convert the line into a sequence of numerical tokens based on the previously created vocabulary. The resulting token list is then iterated over using a for loop. For each iteration, a subsequence, or n-gram, of tokens is extracted, ranging from the beginning of the token list up to the current index ‘i’.

This n-gram sequence represents the input context, with the last token being the target or predicted word. This n-gram sequence is then appended to the ‘input_sequences’ list. This process is repeated for all lines in the text, generating multiple input-output sequences that will be used for training the next word prediction model.

Now let’s pad the input sequences to have equal length:

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding = 'pre'))

In [7]:
input_sequences

array([[   0,    0,    0, ...,    0,    1, 1561],
       [   0,    0,    0, ...,    1, 1561,    5],
       [   0,    0,    0, ..., 1561,    5,  129],
       ...,
       [   0,    0,    0, ...,    1, 8198, 8199],
       [   0,    0,    0, ..., 8198, 8199, 3187],
       [   0,    0,    0, ..., 8199, 3187, 3186]], dtype=int32)

In the above code, the input sequences are padded to ensure all sequences have the same length. The variable ‘max_sequence_len’ is assigned the maximum length among all the input sequences. The ‘pad_sequences’ function is used to pad or truncate the input sequences to match this maximum length.

The ‘pad_sequences’ function takes the input_sequences list, sets the maximum length to ‘max_sequence_len’, and specifies that the padding should be added at the beginning of each sequence using the ‘padding=pre’ argument. Finally, the input sequences are converted into a numpy array to facilitate further processing.

Now let’s split the sequences into input and output:

In [8]:
X= input_sequences[:,:-1]
y = input_sequences[:,-1]

In the above code, the input sequences are split into two arrays, ‘X’ and ‘y’, to create the input and output for training the next word prediction model. The ‘X’ array is assigned the values of all rows in the ‘input_sequences’ array except for the last column. It means that ‘X’ contains all the tokens in each sequence except for the last one, representing the input context.

On the other hand, the ‘y’ array is assigned the values of the last column in the ‘input_sequences’ array, which represents the target or predicted word.

Now let’s convert the output to one-hot encode vectors:

In [9]:
y = np.array(tf.keras.utils.to_categorical(y,num_classes = total_words))

In [10]:
total_words

8200

In the above code, we are converting the output array into a suitable format for training a model, where each target word is represented as a binary vector.

Now let’s build a neural network architecture to train the model:

In [11]:
model = Sequential()
model.add(Embedding(total_words,100, input_length=max_sequence_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words,activation = 'softmax'))
# Explicitly build the model (optional but ensures the model is built)
model.build(input_shape=(None, max_sequence_len - 1))

I0000 00:00:1749587138.215794      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749587138.216484      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [12]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 17, 100)             │         820,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8200)                │       1,238,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,208,800 (8.43 MB)

 Trainable params: 2,208,800 (8.43 MB)

 Non-trainable params: 0 (0.00 B)

None


The code above defines the model architecture for the next word prediction model. The ‘Sequential’ model is created, which represents a linear stack of layers. The first layer added to the model is the ‘Embedding’ layer, which is responsible for converting the input sequences into dense vectors of fixed size. It takes three arguments:

‘total_words’, which represents the total number of distinct words in the vocabulary; 
‘100’, which denotes the dimensionality of the word embeddings; 
and ‘input_length’, which specifies the length of the input sequences.
The next layer added is the ‘LSTM’ layer, a type of recurrent neural network (RNN) layer designed for capturing sequential dependencies in the data. It has 150 units, which means it will learn 150 internal representations or memory cells.

Finally, the ‘Dense’ layer is added, which is a fully connected layer that produces the output predictions. It has ‘total_words’ units and uses the ‘softmax’ activation function to convert the predicted scores into probabilities, indicating the likelihood of each word being the next one in the sequence.

Now let’s compile and train the model:

In [13]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
model.fit(X,y, epochs = 100,verbose= 1)

Epoch 1/100


I0000 00:00:1749587152.440705      94 cuda_dnn.cc:529] Loaded cuDNN version 90300


3010/3010 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.0613 - loss: 6.5721
Epoch 2/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.1128 - loss: 5.6158
Epoch 3/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.1417 - loss: 5.1906
Epoch 4/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.1598 - loss: 4.8527
Epoch 5/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.1757 - loss: 4.5559
Epoch 6/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.1971 - loss: 4.2616
Epoch 7/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.2234 - loss: 3.9785
Epoch 8/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.2491 - loss: 3.7139
Epoch 9/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.2841 - loss: 3.4596
Epoch 10/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.3181 - loss: 3.2370
Epoch 11/100
3010/3010 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.3559 - loss: 3.0170
Epoch 12/100
3010/30

In the above code, the model is being compiled and trained. The ‘compile’ method configures the model for training. The ‘loss’ parameter is set to ‘categorical_crossentropy’, a commonly used loss function for multi-class classification problems. The ‘optimizer’ parameter is set to ‘adam’, an optimization algorithm that adapts the learning rate during training.

The ‘metrics’ parameter is set to ‘accuracy’ to monitor the accuracy during training. After compiling the model, the ‘fit’ method is called to train the model on the input sequences ‘X’ and the corresponding output ‘y’. The ‘epochs’ parameter specifies the number of times the training process will iterate over the entire dataset. The ‘verbose’ parameter is set to ‘1’ to display the training process.

The above code will take more than an hour to execute. Once the code is executed, here’s how we can generate the next word predictions using our model:

In [25]:
seed_text = "My name is"
next_words = 5

In [26]:
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list],maxlen = max_sequence_len - 1, padding = 'pre')
    predicted = np.argmax(model.predict(token_list), axis = -1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
My name is sherlock holmes it is more


The above code generates the next word predictions based on a given seed text. The ‘seed_text’ variable holds the initial text. The ‘next_words’ variable determines the number of predictions to be generated. Inside the for loop, the ‘seed_text’ is converted into a sequence of tokens using the tokenizer. The token sequence is padded to match the maximum sequence length.

The model predicts the next word by calling the ‘predict’ method on the model with the padded token sequence. The predicted word is obtained by finding the word with the highest probability score using ‘np.argmax’. Then, the predicted word is appended to the ‘seed_text’, and the process is repeated for the desired number of ‘next_words’. Finally, the ‘seed_text’ is printed, which contains the initial text followed by the generated predictions.

Next word prediction is a language modelling task in Machine Learning that aims to predict the most probable word or sequence of words that follows a given input context. This task utilizes statistical patterns and linguistic structures to generate accurate predictions based on the context provided. 